# Prerequisites (imports)

In [191]:
import polars as pl # хочу попробовать использовать polars вместо pandas
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

import datetime
from copy import copy

print('Setup completed.')

Setup completed.


# Reading dataset

In [153]:
data = pl.read_csv('train.csv', try_parse_dates=True)
data.head(10)

Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
date,str,f64,f64,f64,f64,f64,str,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str
2014-05-24,"""Penrith""",10.6,24.9,0.0,null,null,"""NW""",20.0,"""SSW""",null,4.0,0.0,68.0,47.0,null,null,null,null,16.7,23.3,"""No""","""No"""
2016-09-04,"""Dartmoor""",6.0,15.1,1.6,null,null,"""NW""",41.0,"""NNW""","""NW""",15.0,24.0,94.0,77.0,1023.0,1024.0,null,null,10.6,13.9,"""Yes""","""No"""
2011-01-26,"""Moree""",24.6,41.4,0.0,12.4,12.8,"""NE""",31.0,"""NNE""","""ENE""",20.0,11.0,51.0,24.0,1012.7,1009.6,0.0,1.0,30.1,39.3,"""No""","""No"""
2013-08-22,"""Bendigo""",3.7,12.6,6.6,1.4,null,"""NW""",41.0,"""WNW""","""NW""",15.0,19.0,95.0,71.0,1011.3,1009.4,8.0,8.0,8.5,12.2,"""Yes""","""Yes"""
2011-10-19,"""Watsonia""",7.6,27.9,0.0,8.0,12.5,"""N""",54.0,"""NE""","""N""",9.0,22.0,57.0,36.0,1026.6,1023.1,1.0,1.0,19.8,27.3,"""No""","""No"""
2014-07-30,"""NorahHead""",12.9,22.8,0.0,null,null,"""NW""",30.0,"""NW""","""WNW""",7.0,9.0,46.0,42.0,1019.0,1015.4,null,null,17.8,21.9,"""No""","""No"""
2016-10-02,"""Hobart""",8.9,18.4,1.0,2.0,4.0,"""N""",48.0,"""NNW""","""NNW""",19.0,11.0,55.0,47.0,1002.9,999.0,5.0,7.0,14.1,17.0,"""No""","""Yes"""
2013-12-08,"""Albany""",13.2,18.3,0.9,3.8,1.0,null,null,"""NW""","""S""",6.0,13.0,77.0,75.0,1017.1,1016.4,7.0,8.0,16.0,16.9,"""No""","""No"""
2013-08-09,"""Bendigo""",4.5,9.6,3.8,1.5,null,"""N""",31.0,"""NNE""","""N""",17.0,17.0,98.0,97.0,1016.2,1012.6,8.0,8.0,7.3,8.7,"""Yes""","""Yes"""


# EDA (working with missing values, feature engineering)

In [154]:
data.shape

(99535, 23)

In [155]:
data.describe()

statistic,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
str,str,str,f64,f64,f64,f64,f64,str,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str
"""count""","""99535""","""99535""",99091.0,99317.0,98569.0,56916.0,51928.0,"""93019""",93059.0,"""92497""","""96880""",98603.0,97702.0,98284.0,96999.0,89721.0,89726.0,61971.0,59529.0,98900.0,97625.0,"""98569""","""99535"""
"""null_count""","""0""","""0""",444.0,218.0,966.0,42619.0,47607.0,"""6516""",6476.0,"""7038""","""2655""",932.0,1833.0,1251.0,2536.0,9814.0,9809.0,37564.0,40006.0,635.0,1910.0,"""966""","""0"""
"""mean""","""2013-04-01 23:46:28.385000""",null,12.176128,23.228321,2.349167,5.49147,7.623223,null,39.958317,null,null,13.988905,18.63839,68.813408,51.482355,1017.655864,1015.259507,4.434881,4.505451,16.980921,21.683034,null,null
"""std""",null,null,6.405731,7.127535,8.486518,4.215723,3.783732,null,13.585303,null,null,8.882297,8.812161,19.082858,20.820645,7.105601,7.037505,2.885424,2.721162,6.505866,6.949604,null,null
"""min""","""2007-11-03""","""Adelaide""",-8.5,-4.1,0.0,0.0,0.0,"""E""",6.0,"""E""","""E""",0.0,0.0,1.0,0.0,980.5,977.1,0.0,0.0,-6.2,-5.1,"""No""","""No"""
"""25%""","""2011-01-07""",null,7.6,17.9,0.0,2.6,4.9,null,31.0,null,null,7.0,13.0,57.0,37.0,1013.0,1010.4,1.0,2.0,12.2,16.6,null,null
"""50%""","""2013-05-27""",null,12.0,22.6,0.0,4.8,8.5,null,39.0,null,null,13.0,19.0,70.0,52.0,1017.6,1015.2,5.0,5.0,16.7,21.1,null,null
"""75%""","""2015-06-13""",null,16.9,28.3,0.6,7.4,10.6,null,48.0,null,null,19.0,24.0,83.0,66.0,1022.4,1020.0,7.0,7.0,21.6,26.4,null,null
"""max""","""2017-06-25""","""Woomera""",31.4,48.1,367.6,145.0,14.3,"""WSW""",135.0,"""WSW""","""WSW""",130.0,83.0,100.0,100.0,1041.0,1039.6,9.0,9.0,40.2,46.2,"""Yes""","""Yes"""


In [156]:
def null_percentages(data: pl.DataFrame) -> None:
    for col in data.columns:
        null_percentage = data.get_column(col).null_count() / data.shape[0] * 100
        print(f'{col}: {null_percentage:.2f}%')
    return

In [157]:
null_percentages(data)

Date: 0.00%
Location: 0.00%
MinTemp: 0.45%
MaxTemp: 0.22%
Rainfall: 0.97%
Evaporation: 42.82%
Sunshine: 47.83%
WindGustDir: 6.55%
WindGustSpeed: 6.51%
WindDir9am: 7.07%
WindDir3pm: 2.67%
WindSpeed9am: 0.94%
WindSpeed3pm: 1.84%
Humidity9am: 1.26%
Humidity3pm: 2.55%
Pressure9am: 9.86%
Pressure3pm: 9.85%
Cloud9am: 37.74%
Cloud3pm: 40.19%
Temp9am: 0.64%
Temp3pm: 1.92%
RainToday: 0.97%
RainTomorrow: 0.00%


In [158]:
columns_to_drop = [
    'Location', # unusable

    # contain too many missing values (~ 40%):
    'Evaporation',
    'Sunshine',
    'Cloud9am',
    'Cloud3pm'
]
data = data.drop(columns_to_drop)

#### 1) Defining season of year

In [159]:
def get_season(date: datetime.date) -> str:
    month = date.month
    if month == 12 or month <= 2:
        return 'Winter'
    if month <= 5:
        return 'Spring'
    if month <= 8:
        return 'Summer'
    # if month <= 11
    return 'Autumn'

In [160]:
Season = data.get_column('Date').map_elements(get_season, return_dtype=pl.String)
data = data.with_columns(Season.to_dummies())

#### 2) Filling missing values in 'Rainfall' and 'WindGustSpeed' columns

In [161]:
data = data.with_columns(
    pl.col('Rainfall').fill_null(strategy='mean'),
    pl.col('WindGustSpeed').fill_null(strategy='mean')
)

#### 3) Working with 'WindGustDir', 'WindDir9am' and 'WindDir3pm' columns

In [162]:
for cat_col in ['WindGustDir', 'WindDir9am', 'WindDir3pm']:
    cat_col_dummies = data.get_column(cat_col).to_dummies()
    data = data.drop(cat_col).with_columns(cat_col_dummies)

In [163]:
null_percentages(data)

Date: 0.00%
MinTemp: 0.45%
MaxTemp: 0.22%
Rainfall: 0.00%
WindGustSpeed: 0.00%
WindSpeed9am: 0.94%
WindSpeed3pm: 1.84%
Humidity9am: 1.26%
Humidity3pm: 2.55%
Pressure9am: 9.86%
Pressure3pm: 9.85%
Temp9am: 0.64%
Temp3pm: 1.92%
RainToday: 0.97%
RainTomorrow: 0.00%
Date_Autumn: 0.00%
Date_Spring: 0.00%
Date_Summer: 0.00%
Date_Winter: 0.00%
WindGustDir_E: 0.00%
WindGustDir_ENE: 0.00%
WindGustDir_ESE: 0.00%
WindGustDir_N: 0.00%
WindGustDir_NE: 0.00%
WindGustDir_NNE: 0.00%
WindGustDir_NNW: 0.00%
WindGustDir_NW: 0.00%
WindGustDir_S: 0.00%
WindGustDir_SE: 0.00%
WindGustDir_SSE: 0.00%
WindGustDir_SSW: 0.00%
WindGustDir_SW: 0.00%
WindGustDir_W: 0.00%
WindGustDir_WNW: 0.00%
WindGustDir_WSW: 0.00%
WindGustDir_null: 0.00%
WindDir9am_E: 0.00%
WindDir9am_ENE: 0.00%
WindDir9am_ESE: 0.00%
WindDir9am_N: 0.00%
WindDir9am_NE: 0.00%
WindDir9am_NNE: 0.00%
WindDir9am_NNW: 0.00%
WindDir9am_NW: 0.00%
WindDir9am_S: 0.00%
WindDir9am_SE: 0.00%
WindDir9am_SSE: 0.00%
WindDir9am_SSW: 0.00%
WindDir9am_SW: 0.00%
WindDi

#### 4) Filling missing values in pairs of dependent columns

In [164]:
def APDMV(
    _data: pl.DataFrame,
    col1_name: str,
    col2_name: str,
    general_prefix: str | None = None
) -> pl.DataFrame:
    '''
    [APDMV - Algorithm for Processing Duo Missing Values]

    Заполняет пропущенные значения в двух взаимосвязанных столбцах, создавая при этом два новых столбца
    по следующему алгоритму:

    Шаг 1.1: Если в какой-то строке пропущены оба значения в колонках col1 и col2, то эти значения заполняются
    нулями, а в колонку both_null ставится значение True;
    
    Шаг 1.2: Иначе пропущенное значение в конкретной колонке (если такое имеется) заполняется средним значением
    из этой колонки, а в колонку both_null ставится значение False;

    Шаг 2: Вне зависимости от того, по какой ветке мы пошли в п. 1, в колонку diff записывается разность
    значения в колонках col2 и col1.

    
    При добавлении колонок both_null и diff в таблицу к их названиям добавляется общий префикс (general_prefix).
    Если же он не был передан отдельно, то общий префикс определяется следующим образом:

    general_prefix = col1_name + '_' + col2_name


    Принимаемые параметры:

    1) _data - таблица (датафрэйм);

    2) col1_name - название первой колонки;

    3) col2_name - название второй колонки;

    4) general_prefix - общий префикс для новых колонок (опционально).


    Возвращаемое значение:
    
    Изменённая таблица (датафрэйм).
    

    Внимание! Функция не гарантирует корректность работы, если ей будут переданы некорректные данные (например,
    колонки, не присутствующие в таблице, или дважды переданный один и тот же общий префикс [это ведёт
    к конфликту названий колонок]).
    '''

    data = copy(_data)

    col1_mean = data.get_column(col1_name).mean()
    col2_mean = data.get_column(col2_name).mean()
    both_null = []
    diff = []

    col1_ind = data.get_column_index(col1_name)
    col2_ind = data.get_column_index(col2_name)
    for row in range(data.shape[0]):
        if data[row, col1_ind] is None and data[row, col2_ind] is None:
            data[row, col1_ind] = data[row, col2_ind] = 0
            both_null.append(True)
        else:
            if data[row, col1_ind] is None:
                data[row, col1_ind] = col1_mean
            if data[row, col2_ind] is None:
                data[row, col2_ind] = col2_mean
            both_null.append(False)
        diff.append(data[row, 2] - data[row, 1])

    if general_prefix is None:
        general_prefix = col1_name + '_' + col2_name
    both_null = pl.Series(both_null, dtype=pl.Boolean).alias(general_prefix + '_' + 'both_null')
    diff = pl.Series(diff, dtype=pl.Float64).alias(general_prefix + '_' + 'diff')
    data = data.with_columns(
        both_null,
        diff
    )

    return data

In [165]:
prefs = [
    'WindSpeed',
    'Humidity',
    'Pressure',
    'Temp'
]
pair_cols = \
    [('MinTemp', 'MaxTemp', 'MinMaxTemp')] + [(pref + '9am', pref + '3pm', pref) for pref in prefs]
pair_cols

[('MinTemp', 'MaxTemp', 'MinMaxTemp'),
 ('WindSpeed9am', 'WindSpeed3pm', 'WindSpeed'),
 ('Humidity9am', 'Humidity3pm', 'Humidity'),
 ('Pressure9am', 'Pressure3pm', 'Pressure'),
 ('Temp9am', 'Temp3pm', 'Temp')]

In [166]:
for col1_name, col2_name, general_prefix in pair_cols:
    data = APDMV(data, col1_name, col2_name, general_prefix)
null_percentages(data)

Date: 0.00%
MinTemp: 0.00%
MaxTemp: 0.00%
Rainfall: 0.00%
WindGustSpeed: 0.00%
WindSpeed9am: 0.00%
WindSpeed3pm: 0.00%
Humidity9am: 0.00%
Humidity3pm: 0.00%
Pressure9am: 0.00%
Pressure3pm: 0.00%
Temp9am: 0.00%
Temp3pm: 0.00%
RainToday: 0.97%
RainTomorrow: 0.00%
Date_Autumn: 0.00%
Date_Spring: 0.00%
Date_Summer: 0.00%
Date_Winter: 0.00%
WindGustDir_E: 0.00%
WindGustDir_ENE: 0.00%
WindGustDir_ESE: 0.00%
WindGustDir_N: 0.00%
WindGustDir_NE: 0.00%
WindGustDir_NNE: 0.00%
WindGustDir_NNW: 0.00%
WindGustDir_NW: 0.00%
WindGustDir_S: 0.00%
WindGustDir_SE: 0.00%
WindGustDir_SSE: 0.00%
WindGustDir_SSW: 0.00%
WindGustDir_SW: 0.00%
WindGustDir_W: 0.00%
WindGustDir_WNW: 0.00%
WindGustDir_WSW: 0.00%
WindGustDir_null: 0.00%
WindDir9am_E: 0.00%
WindDir9am_ENE: 0.00%
WindDir9am_ESE: 0.00%
WindDir9am_N: 0.00%
WindDir9am_NE: 0.00%
WindDir9am_NNE: 0.00%
WindDir9am_NNW: 0.00%
WindDir9am_NW: 0.00%
WindDir9am_S: 0.00%
WindDir9am_SE: 0.00%
WindDir9am_SSE: 0.00%
WindDir9am_SSW: 0.00%
WindDir9am_SW: 0.00%
WindDi

#### 5) Filling missing values in 'RainToday' column and decoding it to pl.Boolean also with 'RainTomorrow' column

In [167]:
data.get_column('RainToday').value_counts()

RainToday,count
str,u32
"""No""",76599
null,966
"""Yes""",21970


In [168]:
data.get_column('RainTomorrow').value_counts()

RainTomorrow,count
str,u32
"""No""",77221
"""Yes""",22314


In [169]:
def yes_no_to_true_false(s: str) -> bool:
    return True if s == 'Yes' else False

In [170]:
data = data.with_columns(
    pl.col('RainToday') \
      .fill_null('No') \
      .map_elements(yes_no_to_true_false, return_dtype=pl.Boolean),
    pl.col('RainTomorrow') \
      .map_elements(yes_no_to_true_false, return_dtype=pl.Boolean)
)

In [171]:
data.head(10)

Date,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Date_Autumn,Date_Spring,Date_Summer,Date_Winter,WindGustDir_E,WindGustDir_ENE,WindGustDir_ESE,WindGustDir_N,WindGustDir_NE,WindGustDir_NNE,WindGustDir_NNW,WindGustDir_NW,WindGustDir_S,WindGustDir_SE,WindGustDir_SSE,WindGustDir_SSW,WindGustDir_SW,WindGustDir_W,WindGustDir_WNW,WindGustDir_WSW,WindGustDir_null,WindDir9am_E,…,WindDir9am_NW,WindDir9am_S,WindDir9am_SE,WindDir9am_SSE,WindDir9am_SSW,WindDir9am_SW,WindDir9am_W,WindDir9am_WNW,WindDir9am_WSW,WindDir9am_null,WindDir3pm_E,WindDir3pm_ENE,WindDir3pm_ESE,WindDir3pm_N,WindDir3pm_NE,WindDir3pm_NNE,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,WindDir3pm_null,MinMaxTemp_both_null,MinMaxTemp_diff,WindSpeed_both_null,WindSpeed_diff,Humidity_both_null,Humidity_diff,Pressure_both_null,Pressure_diff,Temp_both_null,Temp_diff
date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,bool,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,…,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,bool,f64,bool,f64,bool,f64,bool,f64,bool,f64
2014-05-24,10.6,24.9,0.0,20.0,4.0,0.0,68.0,47.0,0.0,0.0,16.7,23.3,false,false,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,false,14.3,false,14.3,false,14.3,true,14.3,false,14.3
2016-09-04,6.0,15.1,1.6,41.0,15.0,24.0,94.0,77.0,1023.0,1024.0,10.6,13.9,true,false,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,false,9.1,false,9.1,false,9.1,false,9.1,false,9.1
2011-01-26,24.6,41.4,0.0,31.0,20.0,11.0,51.0,24.0,1012.7,1009.6,30.1,39.3,false,false,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,false,16.8,false,16.8,false,16.8,false,16.8,false,16.8
2013-08-22,3.7,12.6,6.6,41.0,15.0,19.0,95.0,71.0,1011.3,1009.4,8.5,12.2,true,true,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,false,8.9,false,8.9,false,8.9,false,8.9,false,8.9
2011-10-19,7.6,27.9,0.0,54.0,9.0,22.0,57.0,36.0,1026.6,1023.1,19.8,27.3,false,false,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,false,20.3,false,20.3,false,20.3,false,20.3,false,20.3
2014-07-30,12.9,22.8,0.0,30.0,7.0,9.0,46.0,42.0,1019.0,1015.4,17.8,21.9,false,false,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,…,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,false,9.9,false,9.9,false,9.9,false,9.9,false,9.9
2016-10-02,8.9,18.4,1.0,48.0,19.0,11.0,55.0,47.0,1002.9,999.0,14.1,17.0,false,true,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,false,9.5,false,9.5,false,9.5,false,9.5,false,9.5
2013-12-08,13.2,18.3,0.9,39.958317,6.0,13.0,77.0,75.0,1017.1,1016.4,16.0,16.9,false,false,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,…,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,false,5.1,false,5.1,false,5.1,false,5.1,false,5.1
2013-08-09,4.5,9.6,3.8,31.0,17.0,17.0,98.0,97.0,1016.2,1012.6,7.3,8.7,true,true,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,false,5.1,false,5.1,false,5.1,false,5.1,false,5.1


In [172]:
data.describe()

statistic,Date,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Date_Autumn,Date_Spring,Date_Summer,Date_Winter,WindGustDir_E,WindGustDir_ENE,WindGustDir_ESE,WindGustDir_N,WindGustDir_NE,WindGustDir_NNE,WindGustDir_NNW,WindGustDir_NW,WindGustDir_S,WindGustDir_SE,WindGustDir_SSE,WindGustDir_SSW,WindGustDir_SW,WindGustDir_W,WindGustDir_WNW,WindGustDir_WSW,WindGustDir_null,…,WindDir9am_NW,WindDir9am_S,WindDir9am_SE,WindDir9am_SSE,WindDir9am_SSW,WindDir9am_SW,WindDir9am_W,WindDir9am_WNW,WindDir9am_WSW,WindDir9am_null,WindDir3pm_E,WindDir3pm_ENE,WindDir3pm_ESE,WindDir3pm_N,WindDir3pm_NE,WindDir3pm_NNE,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,WindDir3pm_null,MinMaxTemp_both_null,MinMaxTemp_diff,WindSpeed_both_null,WindSpeed_diff,Humidity_both_null,Humidity_diff,Pressure_both_null,Pressure_diff,Temp_both_null,Temp_diff
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""99535""",99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,…,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0,99535.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""","""2013-04-01 23:46:28.385000""",12.168421,23.213619,2.349167,39.958317,13.903315,18.524352,68.281069,51.084089,918.727466,916.564063,16.940147,21.63097,0.220726,0.224182,0.243482,0.262581,0.252986,0.24095,0.064018,0.056201,0.052273,0.062702,0.049892,0.045301,0.046818,0.056854,0.063093,0.065565,0.062792,0.060572,0.060863,0.068207,0.056905,0.062481,0.065464,…,0.0603,0.060622,0.063897,0.062933,0.052193,0.058432,0.057357,0.050887,0.047883,0.070709,0.058653,0.054745,0.058562,0.060923,0.056915,0.04508,0.055066,0.059758,0.067142,0.0753,0.064199,0.05593,0.06456,0.069272,0.061295,0.065927,0.026674,0.000633,11.045198,0.006118,11.045198,0.007736,11.045198,0.097212,11.045198,0.002401,11.045198
"""std""",null,6.39876,7.143653,8.445236,13.135919,8.907662,8.850797,19.897929,21.042802,301.553745,300.842569,6.538118,6.963938,null,null,0.429186,0.440039,0.434725,0.427662,0.244786,0.230311,0.222578,0.242427,0.217723,0.207964,0.211249,0.231565,0.243132,0.247521,0.24259,0.238544,0.23908,0.252102,0.231661,0.242027,0.247345,…,0.238044,0.238637,0.244571,0.242843,0.222417,0.234559,0.232524,0.219767,0.213519,0.256339,0.234975,0.227482,0.234805,0.239191,0.231681,0.20748,0.22811,0.237039,0.250269,0.263876,0.245107,0.229788,0.245749,0.253918,0.239872,0.248155,0.16113,null,4.978599,null,4.978599,null,4.978599,null,4.978599,null,4.978599
"""min""","""2007-11-03""",-8.5,-4.1,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.2,-5.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-14.876128,0.0,-14.876128,0.0,-14.876128,0.0,-14.876128,0.0,-14.876128
"""25%""","""2011-01-07""",7.6,17.9,0.0,31.0,7.0,13.0,57.0,

In [173]:
null_percentages(data)

Date: 0.00%
MinTemp: 0.00%
MaxTemp: 0.00%
Rainfall: 0.00%
WindGustSpeed: 0.00%
WindSpeed9am: 0.00%
WindSpeed3pm: 0.00%
Humidity9am: 0.00%
Humidity3pm: 0.00%
Pressure9am: 0.00%
Pressure3pm: 0.00%
Temp9am: 0.00%
Temp3pm: 0.00%
RainToday: 0.00%
RainTomorrow: 0.00%
Date_Autumn: 0.00%
Date_Spring: 0.00%
Date_Summer: 0.00%
Date_Winter: 0.00%
WindGustDir_E: 0.00%
WindGustDir_ENE: 0.00%
WindGustDir_ESE: 0.00%
WindGustDir_N: 0.00%
WindGustDir_NE: 0.00%
WindGustDir_NNE: 0.00%
WindGustDir_NNW: 0.00%
WindGustDir_NW: 0.00%
WindGustDir_S: 0.00%
WindGustDir_SE: 0.00%
WindGustDir_SSE: 0.00%
WindGustDir_SSW: 0.00%
WindGustDir_SW: 0.00%
WindGustDir_W: 0.00%
WindGustDir_WNW: 0.00%
WindGustDir_WSW: 0.00%
WindGustDir_null: 0.00%
WindDir9am_E: 0.00%
WindDir9am_ENE: 0.00%
WindDir9am_ESE: 0.00%
WindDir9am_N: 0.00%
WindDir9am_NE: 0.00%
WindDir9am_NNE: 0.00%
WindDir9am_NNW: 0.00%
WindDir9am_NW: 0.00%
WindDir9am_S: 0.00%
WindDir9am_SE: 0.00%
WindDir9am_SSE: 0.00%
WindDir9am_SSW: 0.00%
WindDir9am_SW: 0.00%
WindDi

# Splitting data

In [174]:
target = data.get_column('RainTomorrow')
data = data.drop('RainTomorrow')

In [175]:
train_data, test_data, train_target, test_target = train_test_split(data, target, test_size=0.15)

# Learning XGBClassifier

In [179]:
xgb_model = XGBClassifier(
    max_depth=10,
    objective='binary:logistic',
    random_state=42
)

In [180]:
xgb_model.fit(train_data, train_target)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [181]:
xgb_pred = xgb_model.predict(test_data)
xgb_pred

array([1, 0, 0, ..., 0, 0, 0], shape=(14931,))

# Learning RFC (Random Forest Classifier)

In [188]:
rfc_model = RandomForestClassifier(
    n_estimators=1_000,
    max_depth=10,
    random_state=42
)

In [189]:
rfc_model.fit(train_data, train_target)

RandomForestClassifier(max_depth=10, n_estimators=1000, random_state=42)

In [190]:
rfc_pred = rfc_model.predict(test_data)
rfc_pred

array([False, False, False, ..., False, False, False], shape=(14931,))

# Model's evaluation (validation)

In [197]:
xgb_score = accuracy_score(test_target, xgb_pred) * 100
rfc_score = accuracy_score(test_target, rfc_pred) * 100
print(f'XGBClassifier\'s accuracy is {xgb_score:.2f}%')
print(f'RandomForestClassifier\'s accuracy is {rfc_score:.2f}%')

XGBClassifier's accuracy is 85.02%
RandomForestClassifier's accuracy is 83.40%


# "Feature importances"

In [218]:
xgb_fscores = list(zip(xgbmodel.feature_importances_, data.columns))
xgb_fscores.sort(reverse=True)
print('<---XGBClassifier\'s feature importances--->')
for score, feature in xgb_fscores:
    print(f'{feature} - {score:.5f}')

<---XGBClassifier's feature importances--->
Humidity3pm - 0.07390
WindDir3pm_null - 0.03961
Pressure_both_null - 0.03512
Rainfall - 0.02366
WindDir3pm_NNW - 0.02141
WindGustSpeed - 0.02068
RainToday - 0.01890
Pressure3pm - 0.01787
WindDir3pm_N - 0.01781
WindDir3pm_NNE - 0.01759
WindDir9am_NNE - 0.01759
Date_Winter - 0.01574
WindDir3pm_SSW - 0.01530
WindDir3pm_NE - 0.01487
WindDir3pm_NW - 0.01484
WindGustDir_null - 0.01464
WindDir9am_NE - 0.01444
WindDir9am_N - 0.01443
WindDir3pm_WNW - 0.01408
WindGustDir_NE - 0.01334
Date_Summer - 0.01301
WindGustDir_NNW - 0.01291
WindGustDir_N - 0.01273
WindDir3pm_W - 0.01270
WindDir3pm_ENE - 0.01256
WindDir3pm_S - 0.01253
WindDir9am_SE - 0.01247
WindDir3pm_SSE - 0.01238
WindDir9am_SSW - 0.01234
WindGustDir_SW - 0.01212
WindGustDir_E - 0.01211
WindGustDir_NW - 0.01195
WindGustDir_ESE - 0.01166
WindDir3pm_WSW - 0.01158
WindDir9am_SSE - 0.01143
Date_Autumn - 0.01139
WindDir9am_null - 0.01136
WindDir9am_ENE - 0.01135
Humidity9am - 0.01120
WindGustDir_NNE

In [219]:
rfc_fscores = list(zip(rfcmodel.feature_importances_, data.columns))
rfc_fscores.sort(reverse=True)
print('<---RandomForestClassifier\'s feature importances--->')
for score, feature in rfc_fscores:
    print(f'{feature} - {score:.5f}')

<---RandomForestClassifier's feature importances--->
Humidity3pm - 0.23914
Rainfall - 0.07579
WindSpeed_diff - 0.06089
Humidity_diff - 0.05975
Temp_diff - 0.05947
MinMaxTemp_diff - 0.05922
Pressure_diff - 0.05608
Humidity9am - 0.04880
Pressure3pm - 0.04824
WindGustSpeed - 0.04710
RainToday - 0.04651
Pressure9am - 0.03834
Temp3pm - 0.02364
MaxTemp - 0.01761
MinTemp - 0.01611
Temp9am - 0.01458
WindSpeed3pm - 0.01183
Date - 0.00913
WindSpeed9am - 0.00891
WindDir9am_N - 0.00424
Pressure_both_null - 0.00309
WindDir3pm_N - 0.00238
WindDir3pm_NNW - 0.00225
WindDir9am_NNE - 0.00213
Date_Winter - 0.00193
Date_Summer - 0.00190
WindGustDir_N - 0.00175
WindDir3pm_NW - 0.00167
WindGustDir_NNW - 0.00156
WindGustDir_NW - 0.00136
Date_Autumn - 0.00133
WindDir9am_NNW - 0.00127
WindGustDir_null - 0.00125
Date_Spring - 0.00124
WindDir3pm_WNW - 0.00096
WindDir3pm_NNE - 0.00094
WindDir9am_NW - 0.00094
Humidity_both_null - 0.00089
WindDir3pm_NE - 0.00088
WindDir3pm_W - 0.00088
WindDir3pm_SSE - 0.00086
WindD